In [9]:
import xarray as xr
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import itertools as it
from glob import glob
from netCDF4 import Dataset
from matplotlib.cm import get_cmap
import gc


path_root = '/gpfs/wolf2/cades/cli185/proj-shared/zdr/ERW/output/UQ/20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1/'
path_out = os.path.join(os.environ['PROJDIR'], 'ERW_LDRD', 'results', 'ensemble')

tuples = list(it.product(np.linspace(10, 1000, 10), np.linspace(2, 10, 10)))
tuples = [(1, 0)] + tuples

zsoi = np.array([0.0071, 0.0279, 0.0623, 0.1189, 0.2122, 0.3661, 0.6198, 1.0380, 1.7276, 2.8646])
zisoi = np.array([0.0175, 0.0451, 0.0906, 0.1655, 0.2891, 0.4929, 0.8289, 1.3828, 2.2961, 3.8019])
dzsoi = np.array([0.0175, 0.0276, 0.0455, 0.0750, 0.1236, 0.2038, 0.3360, 0.5539, 0.9133, 1.5058])

# minerals_name = ['Wollastonite_CaSiO3', 'Forsterite_Mg2SiO4', 'Albite_NaAlSi3O8', 
#                 'Anorthite_CaAl2Si2O8', 'Epidote_Ca2FeAl2(SiO4)3(OH)', 'Calcite_CaCO3',
#                 'Labradorite_Ca0.6Na0.4Al1.6Si2.4O8', 'Augite_Ca0.9Mg0.9Na0.1Al0.4Fe0.2Si1.9O6',
#                 'Kfeldspar_KAlSi3O8', 'Enstatite_MgSiO3']
# normal alkali basalt (fast)
pct_basalt = np.array([0, 12, 0, 0, 0, 0, 43, 21, 6, 0])
# normal tholeiitic basalt (slow)
# pct_basalt = np.array([0, 0, 0, 0, 0, 0, 45, 34, 5, 3])

# 1. Foresterite mass evolution over time

In [10]:
# foresterite
primary_mineral_total = pd.DataFrame(
    np.nan, index = range(2000, 2021), columns = range(2, 100)
)
for id in range(1, 38):
    for year in range(2000, 2021):
        nc = Dataset(os.path.join(path_root, f'g{id:05d}', 
            f'20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.{year}-01-01-00000.nc'))
        # g m-2
        ts = np.nansum(nc['primary_mineral'][0, 1, :] * nc['area'][:]) / np.sum(nc['area'][:])
        primary_mineral_total.loc[year, id] = ts
        nc.close()

        gc.collect()

# reference
# foresterite
primary_mineral_ref = pd.Series(
    np.nan, index = range(2000, 2021)
)
for year in range(2000, 2011):
    nc = Dataset(os.path.join(
        os.path.join(os.environ['E3SM_ROOT'], 'output', '20241010_conus_ICB20TRCNPRDCTCBC_erw', 
        'run', f'20241010_conus_ICB20TRCNPRDCTCBC_erw.elm.h1.{year}-01-01-00000.nc')))
    # g m-2
    ts = np.nansum(
        np.sum(nc['primary_mineral_vr_2'][:,:10,:].mean(axis = 0) * \
               dzsoi.reshape(-1, 1), axis = 0) * \
        nc['area'][:]) / np.sum(nc['area'][:])
    primary_mineral_ref.loc[year] = ts
    nc.close()

    gc.collect()

In [11]:
# Temporal drawdown of mineral, in tonne over entire CONUS
cmap = get_cmap('Reds')
clist = [cmap((i+0.5)/100) for i in range(2, 100)]
gra_list = []

fig, axes = plt.subplots(2, 5, figsize = [20, 10], sharex = True, sharey = True)
for id in range(2, 100):
    # one application rate per subplot
    i = np.mod(id - 2, 10)
    ax = axes.flat[i]

    ax.plot(primary_mineral_total.index[1:], 
            primary_mineral_total.loc[2001:, id] / 1000, '-o', color = clist[id-2])

    #ax.plot(primary_mineral_ref.index[1:], primary_mineral_ref.loc[2001:], '-ob')
    if i == 0:
        gra_list.append( tuples[id-1][0] )
    app = tuples[id-1][1]
    ax.set_title(f'App = {app:.2f} (kg m-2 yr-1)')
    ax.set_ylabel('kg m-2')
ax = axes.flat[0]
ax.legend(gra_list, ncol = 5, loc = (0, -1.5))
plt.savefig(os.path.join(path_out, 'primary_mineral.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

/tmp/ipykernel_2754452/1248659328.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap('Reds')


# 2. Dissolution rates over time

Weighted averaged over soil layers and grid cells

In [12]:
r_dissolve = pd.DataFrame(
    np.nan, index = range(2000, 2021), columns = range(2, 100)
)
for id in range(1, 38):
    for year in range(2000, 2021):
        nc = Dataset(os.path.join(path_root, f'g{id:05d}', 
            f'20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.{year}-01-01-00000.nc'))

        # mol m-3 s-1 => mol m-2 s-1
        ts = np.nansum( \
            np.sum(nc['r_dissolve_vr_2'][0,:10,:] * dzsoi.reshape(-1, 1), axis = 0) * \
            nc['area'][:]) / np.sum(nc['area'][:])

        r_dissolve.loc[year, id] = ts

        nc.close()
        gc.collect()

# reference
# foresterite
r_dissolve_ref = pd.Series(
    np.nan, index = range(2000, 2021)
)
for year in range(2000, 2011):
    nc = Dataset(os.path.join(
        os.path.join(os.environ['E3SM_ROOT'], 'output', '20241010_conus_ICB20TRCNPRDCTCBC_erw', 
        'run', f'20241010_conus_ICB20TRCNPRDCTCBC_erw.elm.h1.{year}-01-01-00000.nc')))

    # g m2 * (km2 => m2) => ton
    ts = np.nansum(
        np.sum(
            np.mean(nc['r_dissolve_vr_2'][:,:10,:], axis = 0) * dzsoi.reshape(-1, 1), axis = 0) * \
        nc['area'][:]) / np.sum(nc['area'][:])
    r_dissolve_ref.loc[year] = ts

    nc.close()
    gc.collect()

In [13]:
# Temporal drawdown of mineral, in tonne over entire CONUS
cmap = get_cmap('Reds')
clist = [cmap((i+0.5)/100) for i in range(2, 100)]

fig, axes = plt.subplots(2, 5, figsize = [20, 10], sharex = True, sharey = True)
for id in range(2, 100):
    # one application rate per subplot
    i = np.mod(id-2, 10)
    ax = axes.flat[i]

    # mol m-2 s-1 => mol m-2 yr-1
    ax.plot(r_dissolve.index[1:], 
            r_dissolve.loc[2001:, id] * 86400 * 365, '-o', color = clist[id-2])

    if i == 0:
        gra_list.append( tuples[id-1][0] )
    app = tuples[id-1][1]
    ax.set_title(f'App = {app:.2f} (kg m-2 yr-1)')
    ax.set_ylabel('mol m-2 yr-1')
ax = axes.flat[0]
ax.legend(gra_list, ncol = 5, loc = (0, -1.5))
plt.savefig(os.path.join(path_out, 'r_dissolve.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

/tmp/ipykernel_2754452/4005034771.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap('Reds')


# 3. Spatial map of cation flux due to mineral dissolution

In [30]:
fig, axes = plt.subplots(4, 4, figsize = [20, 20])

for i, id in enumerate(range(1, 17)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.25, 0.25, 41), ncolors=256)

    ax = axes.flat[i]

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        # gC m-2 s-1 => ton ha-1 year-1
        c = hr['primary_cation_flux'][0, 1, :].values * 1e-6 * 1e4 * 86400 * 365, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Primary Mg2+ flux (ton ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'primary_cation_flux.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

In [36]:
fig, axes = plt.subplots(4, 4, figsize = [17, 17])

for i, id in enumerate(range(1, 17)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.25, 0.25, 41), ncolors=256)

    ax = axes.flat[i]

    # gC m-2 s-1 => ton ha-1 year-1
    data = hr['cec_cation_flux_vr_2'][0, :10, :].values * 1e-6 * 1e4 * 86400 * 365
    data = np.sum(data * dzsoi[:10].reshape(-1, 1), axis = 0)

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        c = data, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'CEC Mg2+ flux (ton ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'cec_cation_flux.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

In [32]:
fig, axes = plt.subplots(4, 4, figsize = [17, 17])

for i, id in enumerate(range(1, 17)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.25, 0.25, 41), ncolors=256)

    ax = axes.flat[i]

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        # gC m-2 s-1 => ton ha-1 year-1
        c = hr['background_flux'][0, 1, :].values * 1e-6 * 1e4 * 86400 * 365, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Background Mg2+ flux (ton ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'background_cation_flux.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

In [42]:
fig, axes = plt.subplots(4, 4, figsize = [17, 17])

for i, id in enumerate(range(1, 17)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.25, 0.25, 41), ncolors=256)

    ax = axes.flat[i]

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        # gC m-2 s-1 => ton ha-1 year-1
        c = hr['background_cec'][0, 1, :].values * 1e-6 * 1e4 * 86400 * 365, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Background Mg2+ flux into CEC (ton ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'background_cation_flux_cec.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

In [33]:
fig, axes = plt.subplots(4, 4, figsize = [17, 17])

for i, id in enumerate(range(1, 17)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.25, 0.25, 41), ncolors=256)

    ax = axes.flat[i]

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        # gC m-2 s-1 => ton ha-1 year-1
        c = hr['cation_leached'][0, 1, :].values * 1e-6 * 1e4 * 86400 * 365, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Leached Mg2+ flux (ton ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'leach_cation_flux.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

In [34]:
fig, axes = plt.subplots(4, 4, figsize = [17, 17])

for i, id in enumerate(range(1, 17)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.25, 0.25, 41), ncolors=256)

    ax = axes.flat[i]

    # gC m-2 s-1 => ton ha-1 year-1
    data = hr['cation_infl_vr_2'][0, :10, :].values * 1e-6 * 1e4 * 86400 * 365
    data = np.sum(data * dzsoi[:10].reshape(-1, 1), axis = 0)

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        c = data, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Vertical Mg2+ flux (ton ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'infl_cation_flux.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

In [43]:
# Balance between leaching, cation_infl, and cec flux into the solution
fig, axes = plt.subplots(4, 4, figsize = [17, 17])

for i, id in enumerate(range(1, 17)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.25, 0.25, 41), ncolors=256)

    ax = axes.flat[i]

    # gC m-2 s-1 => ton ha-1 year-1
    data1 = hr['cation_leached'][0, 1, :].values * 1e-6 * 1e4 * 86400 * 365
    data2 = hr['cation_infl_vr_2'][0, :10, :].values * 1e-6 * 1e4 * 86400 * 365
    data2 = np.sum(data2 * dzsoi[:10].reshape(-1, 1), axis = 0)
    data3 = hr['cec_cation_flux_vr_2'][0, :10, :].values * 1e-6 * 1e4 * 86400 * 365
    data3 = np.sum(data3 * dzsoi[:10].reshape(-1, 1), axis = 0)

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        c = data3 + data2 - data1, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Balance between flow out of CEC, vertical drain, and horizontal leaching of Mg2+ flux (ton ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'budget_cation_flux.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

In [26]:
hr = xr.open_dataset(
    os.path.join(path_root, f'g{id:05d}', 
                '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
)
# gC m-2 s-1 => ton C ha-1 s-1
r_benchmark = hr['r_sequestration'][0, :].values * 1e-6 * 1e4 * 86400 * 365
hr.close()

fig, axes = plt.subplots(4, 4, figsize = [17, 17])

for i, id in enumerate(range(2, 18)):
    hr = xr.open_dataset(
        os.path.join(path_root, f'g{id:05d}', 
                    '20241021_conus_ICB20TRCNPRDCTCBC_erw_pft1.elm.h0.2018-01-01-00000.nc')
    )

    norm = BoundaryNorm(boundaries = np.linspace(-0.1, 0.1, 21), ncolors=256)

    ax = axes.flat[i]

    # gC m-2 s-1 => ton C ha-1 s-1
    new_data = hr['r_sequestration'][0, :].values * 1e-6 * 1e4 * 86400 * 365

    cf = ax.scatter(
        x = hr['lon'], y = hr['lat'], s = 11, 
        c = new_data - r_benchmark, 
        cmap = 'Spectral', norm = norm
    )

    gra = tuples[id-1][0]
    app = tuples[id-1][1]
    ax.set_title(f'Grain = {gra}, App = {app:.2f}')

    hr.close()

cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Sequestration (ton C ha-1 yr-1)')
plt.savefig(os.path.join(path_out, 'r_sequestration.png'), dpi = 600., bbox_inches = 'tight')
plt.close(fig)

# 4. Budget diagnostics

In [ ]:
# g m-2 s-1 => g m-2 yr-1
# km2 => m2
# => ton yr-1
background_flux = ((hr['background_flux']*86400*365) * \
                   (hr['area']*1e6) / 1000).sum(axis = 2).sum(axis = 0)
dissolve = ((hr['primary_cation_flux']*86400*365) * \
            (hr['area']*1e6) / 1000).sum(axis = 2).sum(axis = 0)
precip = ((hr['secondary_cation_flux']*86400*365) * \
          (hr['area']*1e6) / 1000).sum(axis = 2).sum(axis = 0)
cec = ((hr['cec_cation_flux']*86400*365) * \
       (hr['area']*1e6) / 1000).sum(axis = 2).sum(axis = 0)
leach = ((hr['cation_leached']*86400*365) * \
          (hr['area']*1e6) / 1000).sum(axis = 2).sum(axis = 0)
infl = []
for cat in range(1,6):
    val = (np.sum(hr[f'cation_infl_vr_{cat}'].values[0, :10, :]*86400*365 * \
                  dzsoi[:10].reshape(-1, 1), axis = 0) * \
           (hr['area']*1e6) / 1000).sum()
    infl.append(float(val))

In [ ]:
fig, axes = plt.subplots(2, 3, figsize = (10, 6))
for i in range(5): # cations
    ax = axes.flat[i]
    ax.bar(x = range(1, 5),
           height = [background_flux[i] - leach[i] - infl[i], dissolve[i], - precip[i], - cec[i]])

In [ ]:
# CONUS benchmark
date_prefix = '20241010'
region = 'conus'
suffix = 'ICB20TRCNPRDCTCBC_erw'
case_name = f'{date_prefix}_{region}_{suffix}'
hr = xr.open_dataset(
    os.path.join(os.environ['E3SM_ROOT'], 'output', 
                 case_name, 'run', f'{case_name}.elm.h1.2001-01-01-00000.nc')
)

data = hr['cation_leached_vr_2'][0, :10, :].mean(axis = 0).values * 86400 * 365
data = np.sum(data * dzsoi.reshape(1, -1, 1), axis = 1)

norm = BoundaryNorm(boundaries = np.linspace(0, 16, 21), ncolors=256)

fig, ax = plt.subplots(figsize = [10, 8])
cf = ax.scatter(
    x = hr['lon'], y = hr['lat'], s = 11, 
    c = data, 
    cmap = 'Spectral', norm = norm
)
cax = fig.add_axes([0.02, 0.1, 0.01, 0.8])
plt.colorbar(cf, cax = cax, label = 'Leached Mg2+ flux (g m-2 yr-1)')

hr.close()